In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder()
from Preprocessing import Preprocessing

In [ ]:
Pre

In [2]:
na_values = ['$', '#VALUE!', '##', 'XNA', '@', '#', 'x', '&']

In [3]:
df = pd.read_csv('./nbfi_vehicle_loan_repayment_dataset/Train_Dataset.csv', na_values=na_values, encoding='utf-8', engine='python')

In [4]:
df[df.select_dtypes(include='int64').columns].isna().sum()

ID                   0
Mobile_Tag           0
Homephone_Tag        0
Workphone_Working    0
Default              0
dtype: int64

ID,고객 대출 신청 ID

Client_Income,고객 소득($)

Car_Owned,다른 차량에 대한 대출을 신청하기 전에 고객이 소유한 모든 차량(0은 아니요, 1은 그렇지 않음을 의미)

Bike_Owned,고객이 소유한 모든 자전거(0은 아니요, 1은 그렇지 않음을 의미)

Active_Loan,대출 신청 당시 진행 중인 다른 대출이 있는지 여부(0은 아니요, 1은 그렇지 않음을 의미)

House_Own,고객이 소유한 주택 수(0은 아니요, 1은 그 외의 주택)

Child_Count,고객이 보유한 자녀 수

Credit_Amount,대출의 신용 금액($)

Loan_Annuity,대출 연금($)

Accompany_Client,고객이 대출을 신청할 때 고객과 동행한 사람

Client_Income_Type,고객 소득 유형

Client_Education,고객이 달성한 최고 교육 수준

Client_Marital_Status,"고객의 결혼 상태(D- 이혼, S- 미혼, M- 기혼, W- 사별)"

Client_Gender,고객의 성별

Loan_Contract_Type,"대출 유형(CL- 현금대출, RL- 리볼빙대출)"

Client_Housing_Type,고객 주택 상황

Population_Region_Relative,고객이 거주하고 있는 지역의 상대적 인구수. 값이 높을수록 고객이 인구가 많은 지역에 거주하고 있음을 의미합니다.

Age_Days,신청서 제출 시점의 고객 나이

Employed_Days,"신청 전, 고객이 수입을 시작한 일수"

Registration_Days,"대출 신청일 전, 고객이 등록을 변경한 일수"

ID_Days,"대출 신청 며칠 전, 고객이 대출을 신청한 신분증을 변경했습니다."

Own_House_Age,고객 소유 주택의 나이(년)

Mobile_Tag,고객이 제공한 휴대폰 번호(1은 예, 0은 아니오를 의미)

Homephone_Tag,고객이 제공한 집전화 번호(1은 예, 0은 아니오를 의미)

Workphone_Working,직장 전화 번호로 연락 가능했는지(1은 예, 0은 아니오)

Client_Occupation,고객 직업 유형

Client_Family_Members,고객이 보유한 가족 구성원 수

Cleint_City_Rating,고객 도시 등급. 3은 최고, 2는 좋음, 1은 평균을 나타냅니다.

Application_Process_Day,고객이 대출을 신청한 요일(0-일,1-월,2-화,3-수,4-목,5-금,6-토)"

Application_Process_Hour,고객이 대출을 신청한 요일 시간

Client_Permanent_Match_Tag,고객 연락처 주소가 영구 주소와 일치하지 않는 경우 표시.

Client_Contact_Work_Tag,고객 직장 주소가 연락처 주소와 일치하지 않는 경우 표시.

Type_Organization,클라이언트가 근무하는 조직 유형

Score_Source_1,다른 출처에서 얻은 점수. 정규화된 점수입니다.

Score_Source_2,다른 출처에서 얻은 점수입니다. 정규화된 점수입니다.

Score_Source_3,다른 소스에서 가져온 점수입니다. 정규화된 점수입니다.

Social_Circle_Default,지난 60일 동안 대출 상환을 불이행한 고객의 친구/가족 수

Phone_Change,"대출 신청 며칠 전에 고객이 휴대폰을 변경했는지"

Credit_Bureau,작년 총 조회 건수

Default,1은 고객이 대출 상환을 불이행했음을 의미하고 0은 그렇지 않음을 의미합니다.

In [5]:
# pay_df.Client_Gender = count_encoder.fit_transform(pay_df.Client_Gender)

pay_df.Client_Gender = label_encoder.fit_transform(pay_df.Client_Gender)

NameError: name 'pay_df' is not defined

In [ ]:
pay_df.Client_Gender

In [ ]:
pay_df.Accompany_Client.value_counts()
# pay_df.Accompany_Client.isna().sum()

In [ ]:
pay_df.Client_Gender.value_counts()

In [ ]:
pay_df.Client_Gender = pay_df.Client_Gender.apply(lambda x : 0 if x == 2 else x)

In [ ]:
pay_df.pivot_table(index=y_target, columns=['Client_Gender'], aggfunc=['mean', 'sum'])

In [ ]:
import time

# posix_timestamp = time.time()
test = 1648712311.982345 + pay_df.Age_Days[6]

In [ ]:
time.gmtime(test)

In [ ]:
pay_df.Age_Days